In [1]:
import time
import random
import numpy as np

import messages
from simulator import Simulator
from agent import Agent
from unity_env import UnityEnv

<class 'sac.sac_agent.SACAgent'>
<class 'sac.sac_env.SACEnv'>


In [7]:
simulator = Simulator(port=5004)
env = UnityEnv(simulator, 9, 2, 1, lambda: np.random.random((2,)))
agent = Agent(env)
#buffer = ReplayBuffer(17, 2, int(1e6))

In [8]:
simulator.start_server()

Accepted connection from <server.Client object at 0x7f80c0e5ec80>


In [57]:
control_message = messages.ControllMessage(0, 10)
message = messages.RequestMessage("step", control_message.to_json())

simulator.server.send([message.to_json()])

response = messages.ResponseMessage.from_json(simulator.server.recive()[0])
observation_message = messages.ObservationMessage.from_json(response.value)
print(observation_message.to_json())


{"agentPostion": {"x": -4.5772385597229, "y": 0.29999998211860657, "z": -4.425022602081299}, "agentRotation": {"x": -2.1524468252209772e-07, "y": 350.4110107421875, "z": 1.7200491697622056e-07}, "velocity": {"x": 7.0524324158327545e-09, "y": -1.4901161193847656e-08, "z": -2.3770390100708028e-08}, "angularVelocity": {"x": -5.719068241205605e-08, "y": 3.0309017517993198e-09, "z": -1.7976729438373695e-08}, "redBallPosition": {"x": 2.3822999000549316, "y": 0.25000008940696716, "z": 2.8051974773406982}, "blueBallPosition": {"x": 0.9334774613380432, "y": -10.0, "z": 2.549999952316284}, "greenBallPosition": {"x": 0.9334774613380432, "y": -10.0, "z": -4.090000152587891}, "grayAreaPosition": {"x": 1.5708506107330322, "y": 0.009999999776482582, "z": 4.366196155548096}, "orangeAreaPosition": {"x": 5.53000020980835, "y": -10.0, "z": 3.6500000953674316}, "whiteAreaPosition": {"x": -6.53000020980835, "y": -10.0, "z": -0.3357459604740143}, "terminate": false, "reward": 0.0}


In [58]:
simulator.server.recive()

KeyboardInterrupt: 

In [11]:
simulator.server.clients[-1].socket.sendall(b"foo")

In [9]:
agent.train()

obs : [0.377196741104126, -0.6436704635620117, 0.0070395999484592015, 0.0, 0.0, 0.5124834060668946, 0.5466055870056152, 0.5073887348175049, -0.5205229759216309]
obs : [0.377196741104126, -0.6436704635620117, 0.007039728429582384, -1.7876868696475868e-08, -1.9797880668193102e-08, 0.5124834060668946, 0.5466055870056152, 0.5073887348175049, -0.5205229759216309]
obs : [0.5104188919067383, -0.5220325469970704, 0.7736818101671007, 0.0, 0.0, 0.6302869319915771, -0.4059588432312012, 0.6302869319915771, -0.4059588432312012]


IndexError: index 1 is out of bounds for axis 0 with size 1

In [ ]:
target_fps = 50
target_frame_duration = 1 / target_fps

max_q_len = 1000
simulator.resume()
time.sleep(0.1)
obs, _, done = simulator.reset()
ep_ret = 0
ep_len = 0

for step in range(1, 1000000+1):
    start_time = time.time()
    if step >= 10000:
        moveInput, turnInput, log_probs = agent.get_action(obs)
    else:
        moveInput, turnInput = random.uniform(-1, 1), random.uniform(-1, 1)

    next_obs, reward, done = simulator.step(moveInput, turnInput)
    ep_ret += reward
    ep_len += 1
    done = False if ep_len==max_q_len else done
    buffer.store(obs, [moveInput, turnInput], reward, next_obs, done)
    obs = next_obs
    
    if done or (ep_len == max_q_len):
        obs, _, done = simulator.reset()
        print(ep_ret)
        ep_ret = 0
        ep_len = 0
        
    #elapsed_time = time.time() - start_time
    #sleep_duration = target_frame_duration - elapsed_time
    #if sleep_duration > 0:
    #    time.sleep(sleep_duration)
        
    if step >= 10000 and step % 50 == 0:
        simulator.pause()
        for _ in range(50):
            batch = buffer.sample_batch()
            agent.update(batch)
            
        simulator.resume()
simulator.pause()

In [ ]:
simulator.shutdown()